In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 회원정보

In [8]:
df1 = pd.read_parquet('../open/train/1.회원정보/201807_train_회원정보.parquet')
df2 = pd.read_parquet('../open/train/1.회원정보/201808_train_회원정보.parquet')
df3 = pd.read_parquet('../open/train/1.회원정보/201809_train_회원정보.parquet')
df4 = pd.read_parquet('../open/train/1.회원정보/201810_train_회원정보.parquet')
df5 = pd.read_parquet('../open/train/1.회원정보/201811_train_회원정보.parquet')
df6 = pd.read_parquet('../open/train/1.회원정보/201812_train_회원정보.parquet')

In [9]:
# 담을 리스트 리스트
all_df = []

# df1~6 담아주기
all_df.append(df1)
all_df.append(df2)
all_df.append(df3)
all_df.append(df4)
all_df.append(df5)
all_df.append(df6)

# merge
merged_df = pd.concat(all_df, ignore_index=True)

print("\n--- 데이터 기본 정보 ---")
merged_df.info()
print("\n")
merged_df.head()


--- 데이터 기본 정보 ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400000 entries, 0 to 2399999
Data columns (total 78 columns):
 #   Column              Dtype  
---  ------              -----  
 0   기준년월                int64  
 1   ID                  object 
 2   남녀구분코드              int64  
 3   연령                  object 
 4   Segment             object 
 5   회원여부_이용가능           int64  
 6   회원여부_이용가능_CA        int64  
 7   회원여부_이용가능_카드론       int64  
 8   소지여부_신용             int64  
 9   소지카드수_유효_신용         int64  
 10  소지카드수_이용가능_신용       int64  
 11  입회일자_신용             int64  
 12  입회경과개월수_신용          int64  
 13  회원여부_연체             int64  
 14  이용거절여부_카드론          int64  
 15  동의여부_한도증액안내         int64  
 16  수신거부여부_TM           int64  
 17  수신거부여부_DM           int64  
 18  수신거부여부_메일           int64  
 19  수신거부여부_SMS          int64  
 20  가입통신회사코드            object 
 21  탈회횟수_누적             int64  
 22  최종탈회후경과월            int64  
 23  탈회횟수_발급6개월이내        int64  
 24  탈회횟수_

,기준년월,ID,남녀구분코드,연령,Segment,회원여부_이용가능,회원여부_이용가능_CA,회원여부_이용가능_카드론,소지여부_신용,소지카드수_유효_신용,...,할인금액_제휴연회비_B0M,청구금액_기본연회비_B0M,청구금액_제휴연회비_B0M,상품관련면제카드수_B0M,임직원면제카드수_B0M,우수회원면제카드수_B0M,기타면제카드수_B0M,카드신청건수,Life_Stage,최종카드발급경과월
0,201807,TRAIN_000000,2,40대,D,1,1,0,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀성장(2),22
1,201807,TRAIN_000001,1,30대,E,1,1,1,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀성장(1),18
2,201807,TRAIN_000002,1,30대,C,1,1,0,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀출산기,20
3,201807,TRAIN_000003,2,40대,D,1,1,0,1,2,...,0,0,0,0개,0개,0개,0개,1,자녀성장(2),17
4,201807,TRAIN_000004,2,40대,E,1,1,1,1,1,...,0,0,0,0개,0개,0개,0개,1,자녀성장(1),15


In [12]:
print("회원정보 기본정보")
print("--- 결측치 ---")
missing_data = merged_df.isnull().sum()
missing_percent = 100 * merged_df.isnull().sum() / len(merged_df)
missing_df = pd.DataFrame({'결측치 수': missing_data, '비율': missing_percent})
missing_df = missing_df[missing_df['결측치 수'] > 0].sort_values(by='비율', ascending=False)
print(missing_df)

# 결측치_많은컬럼 50% 이상
# 논의 후 재조정
high_missing = missing_df[missing_df['비율'] > 50].index.tolist()
if high_missing:
    print(f"\n높은 결측치: {high_missing}")

# 고유값 개수 확인
single_value_cols = []
print("\n--- 고정된 값 확인 ---")
for col in merged_df.columns:
    unique_value_count = merged_df[col].nunique(dropna=False)

    # 고유한 값의 개수가 1개인 경우
    if unique_value_count == 1:
        single_value_cols.append(col)

print(f"\n고정값 : \n{single_value_cols}")


# 컬럼 값이 완전히 동일한 중복 컬럼 찾기
# 병합 후 재실행
print("\n--- 중복 컬럼 확인 ---")
duplicated_cols = {}
cols = merged_df.columns.tolist()
for i in range(len(cols)):
    for j in range(i + 1, len(cols)):
        col1 = cols[i]
        col2 = cols[j]
        if merged_df[col1].equals(merged_df[col2]):
            if col1 not in duplicated_cols:
                duplicated_cols[col1] = []
            duplicated_cols[col1].append(col2)
if duplicated_cols:
    print(f"중복 컬럼들: \n{duplicated_cols}")

회원정보 기본정보
--- 결측치 ---
                 결측치 수         비율
_2순위신용체크구분      958115  39.921458
최종유효년월_신용_이용    534231  22.259625
가입통신회사코드        387570  16.148750
직장시도명           244969  10.207042
최종유효년월_신용_이용가능  210447   8.768625
최종카드발급일자         41965   1.748542
_1순위신용체크구분       27950   1.164583

--- 고정된 값 확인 ---

고정값 : 
['이용카드수_체크_가족', '이용금액_R3M_체크_가족', '연회비할인카드수_B0M', '할인금액_기본연회비_B0M', '할인금액_제휴연회비_B0M', '상품관련면제카드수_B0M', '임직원면제카드수_B0M', '우수회원면제카드수_B0M', '기타면제카드수_B0M']

--- 중복 컬럼 확인 ---
중복 컬럼들: 
{'이용카드수_체크_가족': ['이용금액_R3M_체크_가족', '연회비할인카드수_B0M', '할인금액_기본연회비_B0M', '할인금액_제휴연회비_B0M'], '이용금액_R3M_체크_가족': ['연회비할인카드수_B0M', '할인금액_기본연회비_B0M', '할인금액_제휴연회비_B0M'], '연회비할인카드수_B0M': ['할인금액_기본연회비_B0M', '할인금액_제휴연회비_B0M'], '기본연회비_B0M': ['청구금액_기본연회비_B0M'], '제휴연회비_B0M': ['청구금액_제휴연회비_B0M'], '할인금액_기본연회비_B0M': ['할인금액_제휴연회비_B0M'], '상품관련면제카드수_B0M': ['임직원면제카드수_B0M', '우수회원면제카드수_B0M', '기타면제카드수_B0M'], '임직원면제카드수_B0M': ['우수회원면제카드수_B0M', '기타면제카드수_B0M'], '우수회원면제카드수_B0M': ['기타면제카드수_B0M']}


# 신용정보

In [25]:
df1 = pd.read_parquet('../open/train/2.신용정보/201807_train_신용정보.parquet')
df2 = pd.read_parquet('../open/train/2.신용정보/201808_train_신용정보.parquet')
df3 = pd.read_parquet('../open/train/2.신용정보/201809_train_신용정보.parquet')
df4 = pd.read_parquet('../open/train/2.신용정보/201810_train_신용정보.parquet')
df5 = pd.read_parquet('../open/train/2.신용정보/201811_train_신용정보.parquet')
df6 = pd.read_parquet('../open/train/2.신용정보/201812_train_신용정보.parquet')

In [26]:
# 담을 리스트 리스트
all_df = []

# df1~6 담아주기
all_df.append(df1)
all_df.append(df2)
all_df.append(df3)
all_df.append(df4)
all_df.append(df5)
all_df.append(df6)

# merge
merged_df = pd.concat(all_df, ignore_index=True)

print("\n--- 데이터 기본 정보 ---")
merged_df.info()
print("\n")
merged_df.head()


--- 데이터 기본 정보 ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400000 entries, 0 to 2399999
Data columns (total 42 columns):
 #   Column          Dtype  
---  ------          -----  
 0   기준년월            int64  
 1   ID              object 
 2   최초한도금액          int64  
 3   카드이용한도금액        int64  
 4   CA한도금액          int64  
 5   일시상환론한도금액       int64  
 6   월상환론한도금액        int64  
 7   CA이자율_할인전       float64
 8   CL이자율_할인전       float64
 9   RV일시불이자율_할인전    float64
 10  RV현금서비스이자율_할인전  float64
 11  RV신청일자          float64
 12  RV약정청구율         float64
 13  RV최소결제비율        float64
 14  자발한도감액횟수_R12M   object 
 15  자발한도감액금액_R12M   int64  
 16  자발한도감액후경과월      int64  
 17  강제한도감액횟수_R12M   int64  
 18  강제한도감액금액_R12M   int64  
 19  강제한도감액후경과월      int64  
 20  한도증액횟수_R12M     object 
 21  한도증액금액_R12M     int64  
 22  한도증액후경과월        int64  
 23  상향가능한도금액        int64  
 24  상향가능CA한도금액      int64  
 25  카드론동의여부         object 
 26  월상환론상향가능한도금액    int64  
 27  RV전환가능여부        object

,기준년월,ID,최초한도금액,카드이용한도금액,CA한도금액,일시상환론한도금액,월상환론한도금액,CA이자율_할인전,CL이자율_할인전,RV일시불이자율_할인전,...,연체감액여부_R3M,한도심사요청건수,한도요청거절건수,한도심사요청후경과월,한도심사거절후경과월,시장단기연체여부_R6M,시장단기연체여부_R3M,시장연체상환여부_R6M,시장연체상환여부_R3M,rv최초시작후경과일
0,201807,TRAIN_000000,0,19354,7270,0,0,22.995207,18.254978,17.264967,...,0,0회,0,3,3,0,0,0,0,99999999
1,201807,TRAIN_000001,0,9996,5718,41996,90611,14.793821,14.834873,10.622446,...,0,0회,0,3,3,0,0,0,0,322
2,201807,TRAIN_000002,0,88193,35207,0,0,22.014276,17.875321,17.155829,...,0,0회,0,3,3,0,0,0,0,2378
3,201807,TRAIN_000003,0,19062,6531,0,0,22.998014,22.999453,19.293674,...,0,0회,0,3,3,0,0,0,0,99999999
4,201807,TRAIN_000004,0,177222,47149,48000,155020,14.661948,10.897410,10.654587,...,0,0회,0,3,3,0,0,0,0,99999999


In [27]:
print("회원정보 기본정보")
print("--- 결측치 ---")
missing_data = merged_df.isnull().sum()
missing_percent = 100 * merged_df.isnull().sum() / len(merged_df)
missing_df = pd.DataFrame({'Missing Count': missing_data, 'Missing Percentage': missing_percent})
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values(by='Missing Percentage', ascending=False)
print(missing_df)

# 결측치_많은컬럼 50% 이상
# 논의 후 재조정
high_missing = missing_df[missing_df['Missing Percentage'] > 50].index.tolist()
if high_missing:
    print(f"\n높은 결측치: \n{high_missing}")

# 고유값 개수 확인
single_value_cols = []
print("\n--- 고정된 값 확인 ---")
for col in merged_df.columns:
    unique_value_count = merged_df[col].nunique(dropna=False)

    # 고유한 값의 개수가 1개인 경우
    if unique_value_count == 1:
        single_value_cols.append(col)

print(f"\n고정값 : \n{single_value_cols}")


# 컬럼 값이 완전히 동일한 중복 컬럼 찾기
# 병합 후 재실행
print("\n--- 중복 컬럼 확인 ---")
duplicated_cols = {}
cols = merged_df.columns.tolist()
for i in range(len(cols)):
    for j in range(i + 1, len(cols)):
        col1 = cols[i]
        col2 = cols[j]
        if merged_df[col1].equals(merged_df[col2]):
            if col1 not in duplicated_cols:
                duplicated_cols[col1] = []
            duplicated_cols[col1].append(col2)
if duplicated_cols:
    print(f"중복 컬럼들: \n{duplicated_cols}")

회원정보 기본정보
--- 결측치 ---
          Missing Count  Missing Percentage
RV신청일자          1951236           81.301500
RV전환가능여부          29473            1.228042

높은 결측치: 
['RV신청일자']

--- 고정된 값 확인 ---

고정값 : 
['시장연체상환여부_R3M']

--- 중복 컬럼 확인 ---
